In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

### Draft Year Data

In [2]:
def dfplayerYear(year):
    dfplayerYr=pd.read_csv('Data/draft_year_player/dfplayer'+str(year)+'.csv')
    namePlayer=[]
    
    for elem in dfplayerYr['url']:
        namePlayer.append(elem[11:-5])

    dfplayerYr['name']=namePlayer

    del dfplayerYr['Unnamed: 0']
    del dfplayerYr['url']
    del dfplayerYr['draft']
    del dfplayerYr['shot']
    del dfplayerYr['pos']
    
    return dfplayerYr

In [3]:
fundam = []
for yr in range(1996,2016):
    fundam.append(dfplayerYear(yr))

In [4]:
df_all_fundam = pd.concat(fundam)
df_all_fundam=df_all_fundam.reset_index(drop=True)

df_all_fundam = df_all_fundam[df_all_fundam.height!= 'ml>']
df_all_fundam=df_all_fundam.reset_index(drop=True)
weight_all=df_all_fundam['weight']
height_all=df_all_fundam['height']

df_all_fundam[['weight','height']]=df_all_fundam[['weight','height']].astype(float)

### Height Weight Data

In [5]:
def dfplayerAge(age):
    df_age_yr=pd.read_csv('Data/AgeData/df_age_'+str(age)+'.csv',index_col=0)
    return df_age_yr

In [6]:
df_age_frames=[]
for age in range(19,40):
    df_age_frames.append(dfplayerAge(age))

df_ages = pd.concat(df_age_frames)
df_ages=df_ages.reset_index(drop=True)
del df_ages['age_y']


In [7]:
#calculate goals all 
df_ages['goals_all']=df_ages['goals_ev']+df_ages['goals_pp']+df_ages['goals_sh']

#sort by name and age
df_ages_sorted=df_ages.sort(['name', 'age_x'], ascending=[0, 1])

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [8]:
# reset index
df_ages_sorted=df_ages_sorted.reset_index(drop=True)

df_ages_sorted['goals_all']=df_ages_sorted['goals_all']/df_ages_sorted['games_played']
goals_all_data=list(df_ages_sorted['goals_all'])

# you want to predict the next year data
goals_all_data.pop(0)
goals_all_data.append(0)

df_ages_sorted['goals_all']=goals_all_data
df_ages=df_ages_sorted.groupby("name").apply(lambda x: x.iloc[:-1]).reset_index(drop=True)



### Combined Data (All Data)

In [9]:
df_all = pd.merge(df_ages, df_all_fundam, on='name')
df_all=df_all.drop_duplicates()
df_all['assists_all']=df_all['assists_ev']+df_all['assists_pp']+df_all['assists_sh']

df_all['zs_offense_pct']=df_all['zs_offense_pct']/100.0
df_all['shot_pct']=df_all['shot_pct']/100.0

### convert to per game data

In [10]:
per_games=['goals_ev','goals_pp','goals_sh','assists_all','assists_ev','assists_pp','assists_sh','takeaways',
          'hits','giveaways','blocks','shots_attempted']

for per_game in per_games:
    df_all[per_game]=df_all[per_game]/df_all['games_played']

In [11]:
df_all.head()

,TOT,age_x,assists_ev,assists_pp,assists_sh,blocks,corsi_against,corsi_for,corsi_pct,corsi_rel_pct,...,shots_attempted.1,takeaways,time_on_ice_avg,uniqueseason,zs_defense_pct,zs_offense_pct,goals_all,height,weight,assists_all
0,DET,20.0,0.000000,0.0,0.000000,0.000000,18.0,19.0,51.4,-10.0,...,8.0,0.500000,12.13,1,40.0,0.600,0.000000,188.0,218.0,0.000000
1,DET,21.0,0.000000,0.0,0.000000,0.000000,9.0,22.0,71.0,14.6,...,7.0,0.000000,9.18,1,33.3,0.667,0.060000,188.0,218.0,0.000000
2,DET,22.0,0.060000,0.0,0.000000,0.400000,376.0,409.0,52.1,-2.2,...,141.0,0.140000,10.35,1,33.7,0.663,0.094595,188.0,218.0,0.060000
3,DET,23.0,0.148649,0.0,0.013514,0.527027,628.0,765.0,54.9,-0.4,...,226.0,0.229730,12.18,1,41.2,0.588,0.098765,188.0,218.0,0.162162
4,DET,24.0,0.172840,0.0,0.000000,0.518519,698.0,794.0,53.2,-3.2,...,196.0,0.197531,12.19,1,40.7,0.593,0.208333,188.0,218.0,0.172840


### Normalize Data

In [12]:
df_all_nn=df_all[['goals_all','goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','blocks',
                 'giveaways','hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']]

In [13]:
df_all_nn.head()

,goals_all,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,age_x,blocks,giveaways,hits,shot_pct,shots_attempted,takeaways,zs_offense_pct,height,weight
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,4.000000,20.0,0.000000,0.00000,2.500000,0.000,4.000000,0.500000,0.600,188.0,218.0
1,0.060000,0.000000,0.0,0.0,0.000000,0.0,0.000000,3.500000,21.0,0.000000,0.50000,1.500000,0.000,3.500000,0.000000,0.667,188.0,218.0
2,0.094595,0.060000,0.0,0.0,0.060000,0.0,0.000000,2.820000,22.0,0.400000,0.14000,3.040000,0.038,2.820000,0.140000,0.663,188.0,218.0
3,0.098765,0.094595,0.0,0.0,0.148649,0.0,0.013514,3.054054,23.0,0.527027,0.27027,2.540541,0.054,3.054054,0.229730,0.588,188.0,218.0
4,0.208333,0.098765,0.0,0.0,0.172840,0.0,0.000000,2.419753,24.0,0.518519,0.17284,1.827160,0.066,2.419753,0.197531,0.593,188.0,218.0


In [14]:
cols_to_norm = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','blocks',
                 'giveaways','hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']
df_all_nn=df_all_nn[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [15]:
df_all_nn['age_x'] = df_all_nn['age_x'].apply(lambda x: abs(x))

### Neural Networks Class

In [16]:
#New complete class, with changes:
class Neural_Network(object):
    def __init__(self, Lambda=0):        
        #Define Hyperparameters
        self.inputLayerSize = 19
        self.outputLayerSize = 1
        self.hiddenLayerSize = 10
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
        #Regularization Parameter:
        self.Lambda = Lambda
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)/X.shape[0] + (self.Lambda/2)*(np.sum(self.W1**2)+np.sum(self.W2**2))
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        #Add gradient of regularization term:
        dJdW2 = np.dot(self.a2.T, delta3)/X.shape[0] + self.Lambda*self.W2
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        #Add gradient of regularization term:
        dJdW1 = np.dot(X.T, delta2)/X.shape[0] + self.Lambda*self.W1
        
        return dJdW1, dJdW2
    
    #Helper functions for interacting with other methods/classes
    def getParams(self):
        #Get W1 and W2 Rolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single parameter vector:
        W1_start = 0
        W1_end = self.hiddenLayerSize*self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], \
                             (self.inputLayerSize, self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], \
                             (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [17]:
NN = Neural_Network()

In [18]:
numpyMatrix = df_all_nn.as_matrix()

In [19]:
X = numpyMatrix
data_list = list(df_all['goals_all'])

In [20]:
i=0
y=[]
while i<len(data_list):
    y.append(data_list[i:i+1])
    i=i+1

In [21]:
# epsilon = 1e-4
# x = 1.5

In [22]:
def computeNumericalGradient(N, X, y):
    paramsInitial = N.getParams()
    numgrad = np.zeros(paramsInitial.shape)
    perturb = np.zeros(paramsInitial.shape)
    e = 1e-4

    for p in range(len(paramsInitial)):
        #Set perturbation vector
        perturb[p] = e
        N.setParams(paramsInitial + perturb)
        loss2 = N.costFunction(X, y)

        N.setParams(paramsInitial - perturb)
        loss1 = N.costFunction(X, y)

        #Compute Numerical Gradient
        numgrad[p] = (loss2 - loss1) / (2*e)

        #Return the value we changed to zero:
        perturb[p] = 0

    #Return Params to original value:
    N.setParams(paramsInitial)

    return numgrad 

In [23]:
##Need to modify trainer class a bit to check testing error during training:
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))
        self.testJ.append(self.N.costFunction(self.testX, self.testY))
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = computeNumericalGradient(self.N, X, y)
        
        return cost, grad
        
    def train(self, trainX, trainY, testX, testY):
        #Make an internal variable for the callback function:
        self.X = trainX
        self.y = trainY
        
        self.testX = testX
        self.testY = testY

        #Make empty list to store training costs:
        self.J = []
        self.testJ = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(trainX, trainY), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res


In [28]:
trainX=X[:120]
trainY=y[:120]
testX=X[121:150]
testY=y[121:150]

In [27]:
trainX=X[:int(len(X)*0.8)]
trainY=y[:int(len(y)*0.8)]
testX=X[int(len(X)*0.8):]
testY=y[int(len(y)*0.8):]

In [36]:
#number=0.0001
def Neural_Number(number):
    NN = Neural_Network(Lambda=number)
    #NN = Neural_Network()

    T = trainer(NN)
    T.train(trainX, trainY, testX, testY)

    plt.plot(T.J,label='Train')
    plt.plot(T.testJ,label='Test')
    plt.title("Error with Number of Neighbors ("+str(number)+") ")
    plt.grid(1)
    plt.xlabel('Iterations')
    plt.ylabel('Cost')
    plt.legend(loc='best')
    plt.show()

In [37]:
Neural_Number(0.0001)

Optimization terminated successfully.
         Current function value: 0.004532
         Iterations: 115
         Function evaluations: 116
         Gradient evaluations: 116


KeyboardInterrupt: 

In [85]:
Neural_Number(0.001)

         Current function value: 0.008547
         Iterations: 50
         Function evaluations: 58
         Gradient evaluations: 58


In [86]:
Neural_Number(0.01)

Optimization terminated successfully.
         Current function value: 0.013690
         Iterations: 29
         Function evaluations: 34
         Gradient evaluations: 34


In [87]:
Neural_Number(0.00001)

         Current function value: 0.004152
         Iterations: 50
         Function evaluations: 61
         Gradient evaluations: 61


In [88]:
Neural_Number(0.000001)

         Current function value: 0.003608
         Iterations: 50
         Function evaluations: 59
         Gradient evaluations: 59


In [89]:
Neural_Number(0.0000001)

         Current function value: 0.003433
         Iterations: 50
         Function evaluations: 54
         Gradient evaluations: 54


In [90]:
Neural_Number(0.00000001)

         Current function value: 0.003480
         Iterations: 50
         Function evaluations: 55
         Gradient evaluations: 55
